In [7]:
import pandas as pd
import numpy as np

def dwell(ca1: float, ca2: float, dca: float, sc: float) -> pd.DataFrame:
    # Input validation
    if ca1 >= ca2:
        raise ValueError("ca1 must be less than ca2")
    
    if dca <= 0:
        raise ValueError("dca must be greater than 0")
    
    # Check if (ca2-ca1) is divisible by dca
    if not abs((ca2 - ca1) / dca - round((ca2 - ca1) / dca)) < 1e-10:
        raise ValueError("(ca2-ca1) must be divisible by dca")
    
    # Generate the cam angle series from ca1 to ca2 with dca step
    ca_series = pd.Series(np.arange(ca1, ca2 + dca, dca))
    
    return pd.DataFrame({
        "ca": ca_series,
        "s": sc,  
        "v": 0,   
        "a": 0,   
        "j": 0    
    })

dwell(100, 150, 0.5, 1)

,ca,s,v,a,j
0,100.0,1,0,0,0
1,100.5,1,0,0,0
2,101.0,1,0,0,0
3,101.5,1,0,0,0
4,102.0,1,0,0,0
...,...,...,...,...,...
96,148.0,1,0,0,0
97,148.5,1,0,0,0
98,149.0,1,0,0,0
99,149.5,1,0,0,0


In [9]:
import numpy as np
import pandas as pd

def convel(ca1: float, ca2: float, dca: float, s1: float, s2: float, vc: float, depvar: str) -> pd.DataFrame:
    # Input validation
    if ca1 >= ca2:
        raise ValueError("ca1 must be less than ca2")
    
    if dca <= 0:
        raise ValueError("dca must be greater than 0")
    
    # Check if (ca2-ca1) is divisible by dca
    if not abs((ca2 - ca1) / dca - round((ca2 - ca1) / dca)) < 1e-10:
        raise ValueError("(ca2-ca1) must be divisible by dca")
        
    if depvar not in ['s2', 'vc']:
        raise ValueError("depvar must be either 's2' or 'vc'")
    
    # Generate cam angle series
    ca_series = np.arange(ca1, ca2 + dca, dca)
    
    # Calculate constant velocity based on depvar
    if depvar == 'vc':
        vc = (s2 - s1) / (ca2 - ca1)  

    df = pd.DataFrame({
        "ca": ca_series,
        "v": vc,   
        "a": 0,    
        "j": 0     
    })
    
    # Calculate displacement (s) using cumulative sum
    df["s"] = s1 + (df["ca"] - ca1) * df["v"]
    
    
    # Reorder columns to match required format
    return df[["ca", "s", "v", "a", "j"]] 
    
convel(100, 150, 0.5, 0, 2, 0, 'vc')

,ca,s,v,a,j
0,100.0,0.00,0.04,0,0
1,100.5,0.02,0.04,0,0
2,101.0,0.04,0.04,0,0
3,101.5,0.06,0.04,0,0
4,102.0,0.08,0.04,0,0
...,...,...,...,...,...
96,148.0,1.92,0.04,0,0
97,148.5,1.94,0.04,0,0
98,149.0,1.96,0.04,0,0
99,149.5,1.98,0.04,0,0


In [11]:
import pandas as pd
import numpy as np

def conaccel(ca1: float, ca2: float, dca: float, s1: float, s2: float, 
             v1: float, v2: float, ac: float, depvars: str) -> pd.DataFrame:
    # Input validation
    if ca1 >= ca2:
        raise ValueError("ca2 must be > ca1")
    
    if dca <= 0:
        raise ValueError("dca must be greater than 0")
    
    # Check if (ca2-ca1) is divisible by dca
    if not abs((ca2 - ca1) / dca - round((ca2 - ca1) / dca)) < 1e-10:
        raise ValueError("(ca2-ca1) must be divisible by dca")
        
    if depvars not in ['s2v2', 's2ac', 'v2ac']:
        raise ValueError("depvars must be one of 's2v2', 's2ac', or 'v2ac'")
    
    # Calculate acceleration based on depvars
    if depvars == 's2v2':
        pass
    elif depvars == 's2ac':
        ac = (v2 - v1) / (ca2 - ca1)
    elif depvars == 'v2ac':
        ac = 2 * (s2 - s1 - v1 * (ca2 - ca1)) / ((ca2 - ca1) ** 2)
    
    # Calculate number of points exactly as in MATLAB
    n = round((ca2 - ca1) / dca) + 1
    
    # Initialize arrays with high precision
    df = np.zeros((n, 5), dtype=np.float64)
    
    # Calculate values using MATLAB's precision
    for i in range(n):
        ca = ca1 + dca * i
        t = ca - ca1
        df[i, 0] = ca
        df[i, 1] = s1 + v1 * t + 0.5 * ac * t * t
        df[i, 2] = v1 + ac * t
        df[i, 3] = ac
        df[i, 4] = 0
    
    # Convert to DataFrame
    df = pd.DataFrame(df, columns=['ca', 's', 'v', 'a', 'j'])
    
    # Ensure final values match exactly for s2ac and v2ac modes
    if depvars in ['s2ac', 'v2ac']:
        df.loc[df.index[-1], 's'] = s2
    
    return df.round(6)
    
conaccel(100, 150, 0.5, 0, 3.25, 0.04, 0.09, 0.001, 's2ac')

,ca,s,v,a,j
0,100.0,0.000000,0.0400,0.001,0.0
1,100.5,0.020125,0.0405,0.001,0.0
2,101.0,0.040500,0.0410,0.001,0.0
3,101.5,0.061125,0.0415,0.001,0.0
4,102.0,0.082000,0.0420,0.001,0.0
...,...,...,...,...,...
96,148.0,3.072000,0.0880,0.001,0.0
97,148.5,3.116125,0.0885,0.001,0.0
98,149.0,3.160500,0.0890,0.001,0.0
99,149.5,3.205125,0.0895,0.001,0.0


In [2]:
def dvps2e(cacbr: float, params: dict) -> tuple[float, dict]:
    """
    Calculate lift error for dv-p case (dwell or const velocity to peak)
    
    Args:
        cacbr: relative cam angle vs ca1 at which the accel is cut back (degcm)
        params: dictionary containing:
            S1: lift at start of segment (<lift units>)
            S2: lift at end of segment (<lift units>)
            vr: velocity at start or end of segment (>=0, <lift units>/degcm)
            amx: max accel (<lift units>/degcm^2)
            dmx: max decel (>0, <lift units>/degcm^2)
            jmx: max jerk (<lift units>/degcm^3)
    
    Returns:
        tuple: (s2e, updated_params)
            s2e: lift error at end of segment (%)
            updated_params: dictionary with updated values for:
                dcaa: length of sub-segment A (degcm)
                dcab: length of sub-segment B (degcm)
                dcac: length of sub-segment C (degcm)
                dcad: length of sub-segment D (degcm)
                sab: lift at end of segment A
                sbc: lift at end of segment B
                scd: lift at end of segment C
                dsdcaab: velocity at end of segment A
                dsdcabc: velocity at end of segment B
                dsdcacd: velocity at end of segment C
                d2sdca2bc: accel at end of segment B
    """
    # Check if cacbr exceeds AMX/JMX ratio
    if cacbr > params['amx']/params['jmx']:
        params['dcaa'] = params['amx']/params['jmx']
        params['dcab'] = cacbr - params['dcaa']
        params['d2sdca2bc'] = params['amx']
    else:
        params['dcaa'] = cacbr
        params['dcab'] = 0
        params['d2sdca2bc'] = params['jmx'] * cacbr

    # Calculate segment A end conditions
    params['sab'] = (params['jmx']/6 * params['dcaa']**3 + 
                    params['vr'] * params['dcaa'] + params['S1'])
    params['dsdcaab'] = params['jmx']/2 * params['dcaa']**2 + params['vr']

    # Calculate segment B end conditions
    params['sbc'] = (params['amx']/2 * params['dcab']**2 + 
                    params['dsdcaab'] * params['dcab'] + params['sab'])
    params['dsdcabc'] = params['amx'] * params['dcab'] + params['dsdcaab']

    # Calculate segment C length (minimum of two possible values)
    # Case 1: d2sdca2CD = -DMX (may have segment D)
    dcac1 = (params['dmx'] + params['d2sdca2bc'])/params['jmx']
    # Case 2: dsdcaCD = 0 (no segment D)
    dcac2 = (params['d2sdca2bc']/params['jmx'] + 
             ((params['d2sdca2bc']/params['jmx'])**2 + 
              2*params['dsdcabc']/params['jmx'])**0.5)
    params['dcac'] = min(dcac1, dcac2)

    # Calculate segment C end conditions
    params['scd'] = (-params['jmx']/6 * params['dcac']**3 + 
                    params['d2sdca2bc']/2 * params['dcac']**2 + 
                    params['dsdcabc'] * params['dcac'] + params['sbc'])
    params['dsdcacd'] = (-params['jmx']/2 * params['dcac']**2 + 
                        params['d2sdca2bc'] * params['dcac'] + 
                        params['dsdcabc'])

    # Calculate segment D length
    params['dcad'] = max(0, params['dsdcacd']/params['dmx'])

    # Calculate lift error as percentage
    s2e = ((-params['dmx']/2 * params['dcad']**2 + 
            params['dsdcacd'] * params['dcad'] + 
            params['scd'])/params['S2'] - 1)

    return s2e, params 

In [3]:
from scipy.optimize import root_scalar

def dvpca2e(J: float, params: dict) -> tuple[float, dict]:
    """
    Calculate cam angle error for dv-p segment.
    
    Args:
        J: max jerk (lift units/degcm^3)
        params: dictionary containing:
            ca1: cam angle at start of segment (degcm)
            ca2t: target cam angle at end of segment (degcm)
            dca: cam angle step (degcm)
            amx: max acceleration
            dcaa, dcab, dcac, dcad: segment lengths
    
    Returns:
        tuple: (ca2e, updated_params)
            ca2e: cam angle error at end of segment
            updated_params: dictionary with updated values
    """
    # Store jerk value
    params['jmx'] = J
    
    # Calculate ratio of max accel to jerk
    cacbr = params['amx'] / J
    
    # Calculate lift error
    s2err, params = dvps2e(cacbr, params)
    
    if s2err > 0:
        # Error is positive, search downward
        cacbrmx = cacbr
        cacbrmn = cacbr
        while s2err >= 0:
            cacbrmn = cacbrmn - params['dca']
            s2err, params = dvps2e(cacbrmn, params)
        # Find zero crossing using root_scalar
        sol = root_scalar(lambda x: dvps2e(x, params)[0],  # Only use s2err for root finding
                         bracket=[cacbrmn, cacbrmx],
                         method='brentq')
        cacbr = sol.root
        _, params = dvps2e(cacbr, params)  # Get final params after finding root
        
    elif s2err < 0:
        # Error is negative, search upward
        cacbrmn = cacbr
        cacbrmx = cacbr
        while s2err <= 0:
            cacbrmx = cacbrmx + params['dca']
            s2err, params = dvps2e(cacbrmx, params)
        # Find zero crossing
        sol = root_scalar(lambda x: dvps2e(x, params)[0],  # Only use s2err for root finding
                         bracket=[cacbrmn, cacbrmx],
                         method='brentq')
        cacbr = sol.root
        _, params = dvps2e(cacbr, params)  # Get final params after finding root
    
    # Calculate final cam angle error
    ca2e = (params['ca1'] + params['dcaa'] + params['dcab'] + 
            params['dcac'] + params['dcad'] - params['ca2t'])
    
    return ca2e, params 

In [21]:
import numpy as np
from scipy.optimize import root_scalar


def gcam0_dvp(params: dict) -> np.ndarray:
    """
    Generate cam profile for dv-p (dwell or constant velocity to peak) segment.
    Peak velocity must be greater than starting velocity.
    
    Args:
        params: Dictionary containing all necessary parameters
            ca1, dca, S1, S2, vr, amx, dmx, etc.
    
    Returns:
        np.ndarray: csvaj matrix with columns [ca, s, v, a, j]
    """
    # Handle rise or fall segment
    if params['S2'] > params['S1']:  # rise segment
        # Iterate J<Jmx so that end of segment is an even multiple of dca
        params['ca2t'] = 0
        ca2Jmx, _ = dvpca2e(params['jmx'], params)
        rem = ca2Jmx % params['dca']
        
        if rem == 0:
            J = params['jmx']
            ca2 = ca2Jmx
        else:
            params['ca2t'] = ca2Jmx - rem + params['dca']
            # Find first f where dvpca2e is positive
            for f in np.arange(0.9, 0, -0.1):
                if dvpca2e(f * params['jmx'], params)[0] > 0:
                    break
            
            # Direct equivalent to MATLAB's fzero
            sol = root_scalar(lambda x: dvpca2e(x, params)[0],
                            x0=f * params['jmx'],  # Initial guess
                            x1=params['jmx'],      # Second guess
                            method='secant')       # Using secant method instead of Brentq
            J = sol.root
            
            params['ca2t'] = 0
            ca2, params = dvpca2e(J, params)
        
        if ca2 - params['ca1'] >= 360:
            raise ValueError('required cam angle range >= 360')
        
        # Calculate transition points
        caAB = params['ca1'] + params['dcaa']
        caBC = caAB + params['dcab']
        caCD = caBC + params['dcac']
        
        # Calculate csvaj matrix
        n = round((ca2 - params['ca1']) / params['dca']) + 1
        csvaj = np.zeros((n, 5))  # pre-allocate for compute speed
        
        for i in range(n):
            csvaj[i, 0] = params['ca1'] + params['dca'] * i
            
            if csvaj[i, 0] <= caAB:  # sub-segment A (rise): jerk=J
                carel = csvaj[i, 0] - params['ca1']
                csvaj[i, 1] = J/6 * carel**3 + params['vr'] * carel + params['S1']
                csvaj[i, 2] = J/2 * carel**2 + params['vr']
                csvaj[i, 3] = J * carel
                csvaj[i, 4] = J
                
            elif csvaj[i, 0] <= caBC and params['dcab'] > 0:  # optional sub-segment B (rise): accel=Amx
                carel = csvaj[i, 0] - caAB
                csvaj[i, 1] = params['amx']/2 * carel**2 + params['dsdcaab'] * carel + params['sab']
                csvaj[i, 2] = params['amx'] * carel + params['dsdcaab']
                csvaj[i, 3] = params['amx']
                csvaj[i, 4] = 0
                
            elif csvaj[i, 0] <= caCD or params['dcac'] == 0:  # sub-segment C (rise): jerk=-J
                carel = csvaj[i, 0] - caBC
                csvaj[i, 1] = -J/6 * carel**3 + params['d2sdca2bc']/2 * carel**2 + params['dsdcabc'] * carel + params['sbc']
                csvaj[i, 2] = -J/2 * carel**2 + params['d2sdca2bc'] * carel + params['dsdcabc']
                csvaj[i, 3] = -J * carel + params['d2sdca2bc']
                csvaj[i, 4] = -J
                
            else:  # optional sub-segment D (rise): accel=-Dmx
                carel = csvaj[i, 0] - caCD
                csvaj[i, 1] = -params['dmx']/2 * carel**2 + params['dsdcacd'] * carel + params['scd']
                csvaj[i, 2] = -params['dmx'] * carel + params['dsdcacd']
                csvaj[i, 3] = -params['dmx']
                csvaj[i, 4] = 0
                
    else:  # fall segment: calculate from rise segment symmetry
        # Store original S1, S2 values
        orig_s1, orig_s2 = params['S1'], params['S2']
        
        # Swap S1 and S2 for fall calculation
        params['S1'] = orig_s2
        params['S2'] = orig_s1
        
        # Iterate J<Jmx so that end of segment is an even multiple of dca
        params['ca2t'] = 0
        ca2Jmx, _ = dvpca2e(params['jmx'], params)
        rem = ca2Jmx % params['dca']
        
        if rem == 0:
            J = params['jmx']
            ca2 = ca2Jmx
        else:
            params['ca2t'] = ca2Jmx - rem + params['dca']
            # Find first f where dvpca2e is positive
            for f in np.arange(0.9, 0, -0.1):
                if dvpca2e(f * params['jmx'], params)[0] > 0:
                    break
            
            # Direct equivalent to MATLAB's fzero
            sol = root_scalar(lambda x: dvpca2e(x, params)[0],
                            x0=f * params['jmx'],  # Initial guess
                            x1=params['jmx'],      # Second guess
                            method='secant')       # Using secant method instead of Brentq
            J = sol.root
            
            params['ca2t'] = 0
            ca2, params = dvpca2e(J, params)
            
        if ca2 - params['ca1'] >= 360:
            raise ValueError('required cam angle range >= 360')
            
        # Calculate transition points
        caAB = ca2 - params['dcaa']
        caBC = caAB - params['dcab']
        caCD = caBC - params['dcac']
        
        # Calculate csvaj matrix
        n = round((ca2 - params['ca1']) / params['dca']) + 1
        csvaj = np.zeros((n, 5))  # pre-allocate for compute speed
        
        for i in range(n-1, -1, -1):  # Reverse loop for fall segment
            csvaj[i, 0] = params['ca1'] + params['dca'] * i
            
            if csvaj[i, 0] >= caAB:  # sub-segment A (fall): jerk=-J
                carel = ca2 - csvaj[i, 0]
                csvaj[i, 1] = J/6 * carel**3 + params['vr'] * carel + params['S1']
                csvaj[i, 2] = -(J/2 * carel**2 + params['vr'])
                csvaj[i, 3] = J * carel
                csvaj[i, 4] = -J
                
            elif csvaj[i, 0] >= caBC and params['dcab'] > 0:  # optional sub-segment B (fall): accel=Amx
                carel = caAB - csvaj[i, 0]
                csvaj[i, 1] = params['amx']/2 * carel**2 + params['dsdcaab'] * carel + params['sab']
                csvaj[i, 2] = -(params['amx'] * carel + params['dsdcaab'])
                csvaj[i, 3] = params['amx']
                csvaj[i, 4] = 0
                
            elif csvaj[i, 0] >= caCD or params['dcac'] == 0:  # sub-segment C (fall): jerk=J
                carel = caBC - csvaj[i, 0]
                csvaj[i, 1] = -J/6 * carel**3 + params['d2sdca2bc']/2 * carel**2 + params['dsdcabc'] * carel + params['sbc']
                csvaj[i, 2] = -(-J/2 * carel**2 + params['d2sdca2bc'] * carel + params['dsdcabc'])
                csvaj[i, 3] = -J * carel + params['d2sdca2bc']
                csvaj[i, 4] = J
                
            else:  # optional sub-segment D (fall): accel=-Dmx
                carel = caCD - csvaj[i, 0]
                csvaj[i, 1] = -params['dmx']/2 * carel**2 + params['dsdcacd'] * carel + params['scd']
                csvaj[i, 2] = -(-params['dmx'] * carel + params['dsdcacd'])
                csvaj[i, 3] = -params['dmx']
                csvaj[i, 4] = 0
        
        # Restore original S1, S2 values
        params['S1'], params['S2'] = orig_s1, orig_s2
        
    return csvaj 

In [26]:
# gcam0_dv d-v

import numpy as np
from scipy.optimize import root_scalar

def dvca2e(J: float, params: dict) -> tuple[float, dict]:
    """
    Calculate cam angle error for d-v (dwell to constant velocity) segment.
    
    Args:
        J: max jerk (<lift units>/degcm^3)
        params: Dictionary containing:
            ca1: cam angle at start of segment (degcm)
            ca2t: target cam angle at end of segment (degcm)
            S1: lift at start of segment (<lift units>)
            S2: lift at end of segment (<lift units>)
            vr: velocity at start or end of segment (>=0, <lift units>/degcm)
            amx: max accel (<lift units>/degcm^2)
    
    Returns:
        tuple: (ca2e, params)
            ca2e: cam angle error at end of segment (degcm)
            params: Updated dictionary with calculated parameters
    """
    # Calculate sub-segment lengths and parameters
    if params['vr'] > params['amx']**2 / J:
        params['dcaa'] = params['amx'] / J
        params['dcab'] = params['vr'] / params['amx'] - params['amx'] / J
    else:
        params['dcaa'] = (params['vr'] / J)**0.5
        params['dcab'] = 0
    
    # Calculate positions, velocities, and accelerations at transition points
    params['sab'] = J/6 * params['dcaa']**3 + params['S1']
    params['dsdcaab'] = J/2 * params['dcaa']**2
    
    params['sbc'] = (params['amx']/2 * params['dcab']**2 + 
                     params['dsdcaab'] * params['dcab'] + params['sab'])
    params['dsdcabc'] = params['amx'] * params['dcab'] + params['dsdcaab']
    params['d2sdca2bc'] = J * params['dcaa']
    
    params['dcac'] = params['d2sdca2bc'] / J
    params['scd'] = (-J/6 * params['dcac']**3 + 
                     params['d2sdca2bc']/2 * params['dcac']**2 + 
                     params['dsdcabc'] * params['dcac'] + params['sbc'])
    
    params['dcad'] = (params['S2'] - params['scd']) / params['vr']
    
    # Calculate cam angle error
    ca2e = (params['ca1'] + params['dcaa'] + params['dcab'] + 
            params['dcac'] + params['dcad'] - params['ca2t'])
    
    return ca2e, params 

def gcam0_dv(params: dict) -> np.ndarray:
    """
    Generate cam profile for d-v (dwell to velocity) segment.
    
    Args:
        params: Dictionary containing all necessary parameters
            ca1, dca, S1, S2, vr, amx, dmx, etc.
    
    Returns:
        np.ndarray: csvaj matrix with columns [ca, s, v, a, j]
    """
    # Handle rise or fall segment
    if params['S2'] > params['S1']:  # rise segment
        # Iterate J<Jmx so that end of segment is an even multiple of dca
        params['ca2t'] = 0
        ca2Jmx, _ = dvca2e(params['jmx'], params)
        rem = ca2Jmx % params['dca']
        
        if rem == 0:
            J = params['jmx']
            ca2 = ca2Jmx
        else:
            params['ca2t'] = ca2Jmx - rem + params['dca']
            # Find first f where dvca2e is positive
            for f in np.arange(0.9, 0, -0.1):
                if dvca2e(f * params['jmx'], params)[0] > 0:
                    break
            
            # Direct equivalent to MATLAB's fzero
            sol = root_scalar(lambda x: dvca2e(x, params)[0],
                            x0=f * params['jmx'],
                            x1=params['jmx'],
                            method='secant')
            J = sol.root
            
            params['ca2t'] = 0
            ca2, params = dvca2e(J, params)
        
        if ca2 - params['ca1'] >= 360:
            raise ValueError('required cam angle range >= 360')
        
        # Calculate transition points
        caAB = params['ca1'] + params['dcaa']
        caBC = caAB + params['dcab']
        caCD = caBC + params['dcac']
        
        # Calculate csvaj matrix
        n = round((ca2 - params['ca1']) / params['dca']) + 1
        csvaj = np.zeros((n, 5))  # pre-allocate for compute speed
        
        for i in range(n):
            csvaj[i, 0] = params['ca1'] + params['dca'] * i
            
            if csvaj[i, 0] <= caAB:  # sub-segment A (rise): jerk=J
                carel = csvaj[i, 0] - params['ca1']
                csvaj[i, 1] = J/6 * carel**3 + params['S1']
                csvaj[i, 2] = J/2 * carel**2
                csvaj[i, 3] = J * carel
                csvaj[i, 4] = J
                
            elif csvaj[i, 0] <= caBC and params['dcab'] > 0:  # optional sub-segment B (rise): accel=Amx
                carel = csvaj[i, 0] - caAB
                csvaj[i, 1] = params['amx']/2 * carel**2 + params['dsdcaab'] * carel + params['sab']
                csvaj[i, 2] = params['amx'] * carel + params['dsdcaab']
                csvaj[i, 3] = params['amx']
                csvaj[i, 4] = 0
                
            elif csvaj[i, 0] <= caCD:  # sub-segment C (rise): jerk=-J
                carel = csvaj[i, 0] - caBC
                csvaj[i, 1] = -J/6 * carel**3 + params['d2sdca2bc']/2 * carel**2 + params['dsdcabc'] * carel + params['sbc']
                csvaj[i, 2] = -J/2 * carel**2 + params['d2sdca2bc'] * carel + params['dsdcabc']
                csvaj[i, 3] = -J * carel + params['d2sdca2bc']
                csvaj[i, 4] = -J
                
            else:  # sub-segment D (rise): vel=Vr
                carel = csvaj[i, 0] - caCD
                csvaj[i, 1] = params['vr'] * carel + params['scd']
                csvaj[i, 2] = params['vr']
                csvaj[i, 3] = 0
                csvaj[i, 4] = 0
                
    else:  # fall segment: calculate from rise segment symmetry
        # Swap S1 and S2 for fall calculation
        params['S1'], params['S2'] = params['S2'], params['S1']
        
        # Iterate J<Jmx so that end of segment is an even multiple of dca
        params['ca2t'] = 0
        ca2Jmx, _ = dvca2e(params['jmx'], params)
        rem = ca2Jmx % params['dca']
        
        if rem == 0:
            J = params['jmx']
            ca2 = ca2Jmx
        else:
            params['ca2t'] = ca2Jmx - rem + params['dca']
            for f in np.arange(0.9, 0, -0.1):
                if dvca2e(f * params['jmx'], params)[0] > 0:
                    break
            
            sol = root_scalar(lambda x: dvca2e(x, params)[0],
                            x0=f * params['jmx'],
                            x1=params['jmx'],
                            method='secant')
            J = sol.root
            
            params['ca2t'] = 0
            ca2, params = dvca2e(J, params)
            
        if ca2 - params['ca1'] >= 360:
            raise ValueError('required cam angle range >= 360')
            
        # Calculate transition points
        caAB = ca2 - params['dcaa']
        caBC = caAB - params['dcab']
        caCD = caBC - params['dcac']
        
        # Calculate csvaj matrix
        n = round((ca2 - params['ca1']) / params['dca']) + 1
        csvaj = np.zeros((n, 5))  # pre-allocate for compute speed
        
        for i in range(n-1, -1, -1):  # Reverse loop for fall segment
            csvaj[i, 0] = params['ca1'] + params['dca'] * i
            
            if csvaj[i, 0] >= caAB:  # sub-segment A (fall): jerk=-J
                carel = ca2 - csvaj[i, 0]
                csvaj[i, 1] = J/6 * carel**3 + params['S1']
                csvaj[i, 2] = -J/2 * carel**2
                csvaj[i, 3] = J * carel
                csvaj[i, 4] = -J
                
            elif csvaj[i, 0] >= caBC and params['dcab'] > 0:  # optional sub-segment B (fall): accel=Amx
                carel = caAB - csvaj[i, 0]
                csvaj[i, 1] = params['amx']/2 * carel**2 + params['dsdcaab'] * carel + params['sab']
                csvaj[i, 2] = -(params['amx'] * carel + params['dsdcaab'])
                csvaj[i, 3] = params['amx']
                csvaj[i, 4] = 0
                
            elif csvaj[i, 0] >= caCD:  # sub-segment C (fall): jerk=J
                carel = caBC - csvaj[i, 0]
                csvaj[i, 1] = -J/6 * carel**3 + params['d2sdca2bc']/2 * carel**2 + params['dsdcabc'] * carel + params['sbc']
                csvaj[i, 2] = -(-J/2 * carel**2 + params['d2sdca2bc'] * carel + params['dsdcabc'])
                csvaj[i, 3] = -J * carel + params['d2sdca2bc']
                csvaj[i, 4] = J
                
            else:  # sub-segment D (fall): vel=-Vr
                carel = caCD - csvaj[i, 0]
                csvaj[i, 1] = params['vr'] * carel + params['scd']
                csvaj[i, 2] = -params['vr']
                csvaj[i, 3] = 0
                csvaj[i, 4] = 0
        
        # Restore original S1, S2 values (not needed as per our previous discussion)
        params['S1'], params['S2'] = params['S2'], params['S1']
        
    return csvaj 

In [30]:
#gcam0_dvsva dv-sva

import numpy as np
from typing import Tuple

def dvsvas2e(cacbr: float, params: dict) -> Tuple[float, dict]:
    """
    Calculate lift error for dv-sva (dwell or constant velocity to specified velocity and acceleration) segment.
    
    Args:
        cacbr: relative cam angle vs ca1 at which the accel is cut back (degcm)
        params: Dictionary containing:
            S1: lift at start of segment (<lift units>)
            S2: lift at end of segment (<lift units>)
            vr: velocity at start or end of segment (>=0, <lift units>/degcm)
            vmatch: final velocity (>=0, <lift units>/degcm)
            amatch: final accel (=>0, <lift units>/degcm^2)
            amx: max accel (<lift units>/degcm^2)
            dmx: max decel (>0, <lift units>/degcm^2)
            jmx: jerk (<lift units>/degcm^3)
    
    Returns:
        tuple: (s2e, params)
            s2e: lift error at end of segment (%)
            params: Updated dictionary with calculated parameters
    """
    # Calculate sub-segment A and B parameters
    if cacbr > params['amx'] / params['jmx']:
        params['dcaa'] = params['amx'] / params['jmx']
        params['dcab'] = cacbr - params['dcaa']
        params['d2sdca2bc'] = params['amx']
    else:
        params['dcaa'] = cacbr
        params['dcab'] = 0
        params['d2sdca2bc'] = params['jmx'] * cacbr
    
    # Calculate positions and velocities at transition points
    params['sab'] = params['jmx']/6 * params['dcaa']**3 + params['vr'] * params['dcaa'] + params['S1']
    params['dsdcaab'] = params['jmx']/2 * params['dcaa']**2 + params['vr']
    params['sbc'] = (params['amx']/2 * params['dcab']**2 + 
                     params['dsdcaab'] * params['dcab'] + params['sab'])
    params['dsdcabc'] = params['amx'] * params['dcab'] + params['dsdcaab']
    
    # Choose dcad to match final velocity to vmatch
    # dcad = length of segment D or negative for partial segment C with d2sdca2de > -dmx (degcm)
    a = params['jmx']
    b = 2 * params['dmx']
    c = (-params['vr'] - params['d2sdca2bc'] * params['dcab'] - 
         (params['d2sdca2bc']**2 + 0.5 * params['amatch']**2 - params['dmx']**2) / params['jmx'] + 
         params['vmatch'])
    
    dcad = -c / params['dmx']  # dcac > 0 result
    if dcad > 0:
        params['dcad'] = dcad
        params['d2sdca2de'] = -params['dmx']
    else:
        arg = b**2 - 4*a*c
        if arg < 0:
            print("Warning: gcam0 function, 'dv-sva' option: Unable to match v2 for current dcaa iteration. Continuing with d2sdca2de=0.")
        
        # dcad < 0 result: physical root for continuity with dcac>0 result
        # b^2-4*a*c=0 corresponds to min dcad=-dmx/jmx for d2sdca2de = 0
        dcad = (-b + np.sqrt(max(arg, 0))) / (2*a)
        params['dcad'] = 0
        params['d2sdca2de'] = -params['dmx'] - dcad * params['jmx']
    
    # Calculate remaining parameters
    params['dcac'] = (params['d2sdca2bc'] - params['d2sdca2de']) / params['jmx']
    params['scd'] = (-params['jmx']/6 * params['dcac']**3 + 
                     params['d2sdca2bc']/2 * params['dcac']**2 + 
                     params['dsdcabc'] * params['dcac'] + params['sbc'])
    params['dsdcacd'] = (-params['jmx']/2 * params['dcac']**2 + 
                         params['d2sdca2bc'] * params['dcac'] + params['dsdcabc'])
    params['sde'] = (-params['dmx']/2 * params['dcad']**2 + 
                     params['dsdcacd'] * params['dcad'] + params['scd'])
    params['dsdcade'] = -params['dmx'] * params['dcad'] + params['dsdcacd']
    params['dcae'] = (params['amatch'] - params['d2sdca2de']) / params['jmx']
    
    # Calculate lift error
    s2e = ((params['jmx']/6 * params['dcae']**3 + 
            params['d2sdca2de']/2 * params['dcae']**2 + 
            params['dsdcade'] * params['dcae'] + params['sde']) / params['S2'] - 1)
    
    return s2e, params 

from scipy.optimize import root_scalar


def dvsvaca2e(J: float, params: dict) -> tuple[float, dict]:
    """
    Calculate cam angle error for dv-sva (dwell or constant velocity to specified velocity and acceleration) segment.
    
    Args:
        J: max jerk (lift units/degcm^3)
        params: Dictionary containing:
            ca1: cam angle at start of segment (degcm)
            ca2t: target cam angle at end of segment (degcm)
            dca: cam angle step (degcm): typically 0.5 or 1.0
            dcaa: length of sub-segment A (degcm)
            dcab: length of sub-segment B (degcm)
            dcac: length of sub-segment C (degcm)
            dcad: length of sub-segment D (degcm)
            dcae: length of sub-segment E (degcm)
            amx: max accel limit (<lift units>/degcm^2)
    
    Returns:
        tuple: (ca2e, params)
            ca2e: cam angle error at end of segment (degcm)
            params: Updated dictionary with calculated parameters
    """
    # Store J for use in dvsvas2e
    params['jmx'] = J
    
    # Calculate initial cacbr (cam angle change before reversal)
    cacbr = params['amx'] / J
    
    # Get initial s2err
    s2err, params = dvsvas2e(cacbr, params)
    
    if s2err > 0:
        cacbrmx = cacbr
        cacbrmn = cacbr
        while s2err >= 0:
            cacbrmn = cacbrmn - params['dca']
            s2err, params = dvsvas2e(cacbrmn, params)
        
        # Find zero crossing
        sol = root_scalar(lambda x: dvsvas2e(x, params)[0],
                         bracket=[cacbrmn, cacbrmx],
                         method='brentq')
        cacbr = sol.root
        
    elif s2err < 0:
        cacbrmn = cacbr
        cacbrmx = cacbr
        while s2err <= 0:
            cacbrmx = cacbrmx + params['dca']
            s2err, params = dvsvas2e(cacbrmx, params)
        
        # Find zero crossing
        sol = root_scalar(lambda x: dvsvas2e(x, params)[0],
                         bracket=[cacbrmn, cacbrmx],
                         method='brentq')
        cacbr = sol.root
    
    # Calculate final cam angle error
    ca2e = (params['ca1'] + params['dcaa'] + params['dcab'] + 
            params['dcac'] + params['dcad'] + params['dcae'] - params['ca2t'])
    
    return ca2e, params 


import numpy as np
from scipy.optimize import root_scalar


def gcam0_dvsva(params: dict) -> np.ndarray:
    """
    Generate cam profile for dv-sva (dwell or constant velocity to specified velocity and acceleration) segment.
    
    Args:
        params: Dictionary containing all necessary parameters
            ca1, dca, S1, S2, vr, amx, dmx, etc.
    
    Returns:
        np.ndarray: csvaj matrix with columns [ca, s, v, a, j]
    """
    # Handle rise or fall segment
    if params['S2'] > params['S1']:  # rise segment
        # Iterate J<Jmx so that end of segment is an even multiple of dca
        params['ca2t'] = 0
        ca2Jmx, _ = dvsvaca2e(params['jmx'], params)
        rem = ca2Jmx % params['dca']
        
        if rem == 0:
            J = params['jmx']
            ca2 = ca2Jmx
        else:
            params['ca2t'] = ca2Jmx - rem + params['dca']
            # Find first f where dvsvaca2e is positive
            for f in np.arange(0.9, 0, -0.1):
                if dvsvaca2e(f * params['jmx'], params)[0] > 0:
                    break
            
            # Direct equivalent to MATLAB's fzero
            sol = root_scalar(lambda x: dvsvaca2e(x, params)[0],
                            x0=f * params['jmx'],
                            x1=params['jmx'],
                            method='secant')
            J = sol.root
            
            params['ca2t'] = 0
            ca2, params = dvsvaca2e(J, params)
        
        if ca2 - params['ca1'] >= 360:
            raise ValueError('required cam angle range >= 360')
        
        # Calculate transition points
        caAB = params['ca1'] + params['dcaa']
        caBC = caAB + params['dcab']
        caCD = caBC + params['dcac']
        caDE = caCD + params['dcad']
        
        # Calculate csvaj matrix
        n = round((ca2 - params['ca1']) / params['dca']) + 1
        csvaj = np.zeros((n, 5))  # pre-allocate for compute speed
        
        for i in range(n):
            csvaj[i, 0] = params['ca1'] + params['dca'] * i
            
            if csvaj[i, 0] <= caAB:  # sub-segment A (rise): jerk=J
                carel = csvaj[i, 0] - params['ca1']
                csvaj[i, 1] = J/6 * carel**3 + params['vr'] * carel + params['S1']
                csvaj[i, 2] = J/2 * carel**2 + params['vr']
                csvaj[i, 3] = J * carel
                csvaj[i, 4] = J
                
            elif csvaj[i, 0] <= caBC and params['dcab'] > 0:  # optional sub-segment B (rise): accel=Amx
                carel = csvaj[i, 0] - caAB
                csvaj[i, 1] = params['amx']/2 * carel**2 + params['dsdcaab'] * carel + params['sab']
                csvaj[i, 2] = params['amx'] * carel + params['dsdcaab']
                csvaj[i, 3] = params['amx']
                csvaj[i, 4] = 0
                
            elif csvaj[i, 0] <= caCD:  # sub-segment C (rise): jerk=-J
                carel = csvaj[i, 0] - caBC
                csvaj[i, 1] = -J/6 * carel**3 + params['d2sdca2bc']/2 * carel**2 + params['dsdcabc'] * carel + params['sbc']
                csvaj[i, 2] = -J/2 * carel**2 + params['d2sdca2bc'] * carel + params['dsdcabc']
                csvaj[i, 3] = -J * carel + params['d2sdca2bc']
                csvaj[i, 4] = -J
                
            elif csvaj[i, 0] <= caDE and params['dcad'] > 0:  # optional sub-segment D (rise): accel=-Dmx
                carel = csvaj[i, 0] - caCD
                csvaj[i, 1] = -params['dmx']/2 * carel**2 + params['dsdcacd'] * carel + params['scd']
                csvaj[i, 2] = -params['dmx'] * carel + params['dsdcacd']
                csvaj[i, 3] = -params['dmx']
                csvaj[i, 4] = 0
                
            else:  # optional sub-segment E (rise): jerk=J
                carel = csvaj[i, 0] - caDE
                csvaj[i, 1] = J/6 * carel**3 + params['d2sdca2de']/2 * carel**2 + params['dsdcade'] * carel + params['sde']
                csvaj[i, 2] = J/2 * carel**2 + params['d2sdca2de'] * carel + params['dsdcade']
                csvaj[i, 3] = J * carel + params['d2sdca2de']
                csvaj[i, 4] = J
                
    else:  # fall segment: calculate from rise segment symmetry
        # Swap S1 and S2 for fall calculation
        params['S1'], params['S2'] = params['S2'], params['S1']
        
        # Iterate J<Jmx so that end of segment is an even multiple of dca
        params['ca2t'] = 0
        ca2Jmx, _ = dvsvaca2e(params['jmx'], params)
        rem = ca2Jmx % params['dca']
        
        if rem == 0:
            J = params['jmx']
            ca2 = ca2Jmx
        else:
            params['ca2t'] = ca2Jmx - rem + params['dca']
            for f in np.arange(0.9, 0, -0.1):
                if dvsvaca2e(f * params['jmx'], params)[0] > 0:
                    break
            
            sol = root_scalar(lambda x: dvsvaca2e(x, params)[0],
                            x0=f * params['jmx'],
                            x1=params['jmx'],
                            method='secant')
            J = sol.root
            
            params['ca2t'] = 0
            ca2, params = dvsvaca2e(J, params)
        
        if ca2 - params['ca1'] >= 360:
            raise ValueError('required cam angle range >= 360')
        
        # Calculate transition points for fall segment
        caAB = ca2 - params['dcaa']
        caBC = caAB - params['dcab']
        caCD = caBC - params['dcac']
        caDE = caCD - params['dcad']
        
        # Calculate csvaj matrix
        n = round((ca2 - params['ca1']) / params['dca']) + 1
        csvaj = np.zeros((n, 5))  # pre-allocate for compute speed
        
        for i in range(n-1, -1, -1):  # Reverse loop for fall segment
            csvaj[i, 0] = params['ca1'] + params['dca'] * i
            
            if csvaj[i, 0] >= caAB:  # sub-segment A (fall): jerk=-J
                carel = ca2 - csvaj[i, 0]
                csvaj[i, 1] = J/6 * carel**3 + params['vr'] * carel + params['S1']
                csvaj[i, 2] = -(J/2 * carel**2 + params['vr'])
                csvaj[i, 3] = J * carel
                csvaj[i, 4] = -J
                
            elif csvaj[i, 0] >= caBC and params['dcab'] > 0:  # optional sub-segment B (fall): accel=Amx
                carel = caAB - csvaj[i, 0]
                csvaj[i, 1] = params['amx']/2 * carel**2 + params['dsdcaab'] * carel + params['sab']
                csvaj[i, 2] = -(params['amx'] * carel + params['dsdcaab'])
                csvaj[i, 3] = params['amx']
                csvaj[i, 4] = 0
                
            elif csvaj[i, 0] >= caCD:  # sub-segment C (fall): jerk=J
                carel = caBC - csvaj[i, 0]
                csvaj[i, 1] = -J/6 * carel**3 + params['d2sdca2bc']/2 * carel**2 + params['dsdcabc'] * carel + params['sbc']
                csvaj[i, 2] = -(-J/2 * carel**2 + params['d2sdca2bc'] * carel + params['dsdcabc'])
                csvaj[i, 3] = -J * carel + params['d2sdca2bc']
                csvaj[i, 4] = J
                
            elif csvaj[i, 0] >= caDE and params['dcad'] > 0:  # optional sub-segment D (fall): accel=-Dmx
                carel = caCD - csvaj[i, 0]
                csvaj[i, 1] = -params['dmx']/2 * carel**2 + params['dsdcacd'] * carel + params['scd']
                csvaj[i, 2] = -(-params['dmx'] * carel + params['dsdcacd'])
                csvaj[i, 3] = -params['dmx']
                csvaj[i, 4] = 0
                
            else:  # optional sub-segment E (fall): jerk=-J
                carel = caDE - csvaj[i, 0]
                csvaj[i, 1] = J/6 * carel**3 + params['d2sdca2de']/2 * carel**2 + params['dsdcade'] * carel + params['sde']
                csvaj[i, 2] = -(J/2 * carel**2 + params['d2sdca2de'] * carel + params['dsdcade'])
                csvaj[i, 3] = J * carel + params['d2sdca2de']
                csvaj[i, 4] = -J
        
        # Restore original S1, S2 values
        params['S1'], params['S2'] = params['S2'], params['S1']
    
    return csvaj 

In [33]:
# dv-d

import numpy as np

def dvds2e(cacbr: float, params: dict) -> tuple[float, dict]:
    """
    Calculate lift error for dv-d (dwell or constant velocity to dwell) segment.
    
    Args:
        cacbr: relative cam angle vs ca1 at which the accel is cut back (degcm)
        params: Dictionary containing:
            S1: lift at start of segment (<lift units>)
            S2: lift at end of segment (<lift units>)
            vr: velocity at start or end of segment (>=0, <lift units>/degcm)
            amx: max accel (<lift units>/degcm^2)
            dmx: max decel (>0, <lift units>/degcm^2)
            jmx: jerk (<lift units>/degcm^3)
    
    Returns:
        tuple: (s2e, params)
            s2e: lift error at end of segment (%)
            params: Updated dictionary with calculated parameters:
                dcaa: length of sub-segment A (degcm)
                dcab: length of sub-segment B (degcm)
                dcac: length of sub-segment C (degcm)
                dcad: length of sub-segment D (degcm)
                dcae: length of sub-segment E (degcm)
                sab: lift at end of segment A (<lift units>)
                sbc: lift at end of segment B (<lift units>)
                scd: lift at end of segment C (<lift units>)
                sde: lift at end of segment D (<lift units>)
                dsdcaab: velocity at AB transition (<lift units>/degcm)
                dsdcabc: velocity at BC transition (<lift units>/degcm)
                dsdcacd: velocity at CD transition (<lift units>/degcm)
                dsdcade: velocity at DE transition (<lift units>/degcm)
                d2sdca2bc: accel at BC transition (<lift units>/degcm^2)
                d2sdca2de: accel at DE transition (<lift units>/degcm^2)
    """
    # Calculate sub-segment A and B parameters
    if cacbr > params['amx'] / params['jmx']:
        params['dcaa'] = params['amx'] / params['jmx']
        params['dcab'] = cacbr - params['dcaa']
        params['d2sdca2bc'] = params['amx']
    else:
        params['dcaa'] = cacbr
        params['dcab'] = 0
        params['d2sdca2bc'] = params['jmx'] * cacbr
    
    # Calculate positions and velocities at transition points
    params['sab'] = params['jmx']/6 * params['dcaa']**3 + params['vr'] * params['dcaa'] + params['S1']
    params['dsdcaab'] = params['jmx']/2 * params['dcaa']**2 + params['vr']
    params['sbc'] = (params['amx']/2 * params['dcab']**2 + 
                     params['dsdcaab'] * params['dcab'] + params['sab'])
    params['dsdcabc'] = params['amx'] * params['dcab'] + params['dsdcaab']
    
    # Calculate sub-segment C parameters
    # d2sdca2CD=-DMX (may have segment D)
    dcac1 = (params['dmx'] + params['d2sdca2bc']) / params['jmx']
    # dsdca2=0 without segment D
    dcac2 = (params['d2sdca2bc'] / params['jmx'] + 
             np.sqrt(0.5 * (params['d2sdca2bc'] / params['jmx'])**2 + 
                    params['dsdcabc'] / params['jmx']))
    params['dcac'] = min(dcac1, dcac2)
    
    # Calculate positions and velocities for segment C
    params['scd'] = (-params['jmx']/6 * params['dcac']**3 + 
                     params['d2sdca2bc']/2 * params['dcac']**2 + 
                     params['dsdcabc'] * params['dcac'] + params['sbc'])
    params['dsdcacd'] = (-params['jmx']/2 * params['dcac']**2 + 
                         params['d2sdca2bc'] * params['dcac'] + params['dsdcabc'])
    
    # Calculate sub-segment D and E parameters
    if dcac1 < dcac2:
        params['dcad'] = params['dsdcacd'] / params['dmx'] - params['dmx'] / (2 * params['jmx'])
        params['d2sdca2de'] = -params['dmx']
    else:
        params['dcad'] = 0
        params['d2sdca2de'] = -params['jmx'] * params['dcac'] + params['d2sdca2bc']
    
    # Calculate final positions and velocities
    params['sde'] = (-params['dmx']/2 * params['dcad']**2 + 
                     params['dsdcacd'] * params['dcad'] + params['scd'])
    params['dsdcade'] = -params['dmx'] * params['dcad'] + params['dsdcacd']
    params['dcae'] = -params['d2sdca2de'] / params['jmx']
    
    # Calculate lift error
    s2e = ((params['jmx']/6 * params['dcae']**3 + 
            params['d2sdca2de']/2 * params['dcae']**2 + 
            params['dsdcade'] * params['dcae'] + params['sde']) / params['S2'] - 1)
    
    return s2e, params 

from scipy.optimize import root_scalar


def dvdca2e(J: float, params: dict) -> tuple[float, dict]:
    """
    Calculate cam angle error for dv-d (dwell or constant velocity to dwell) segment.
    
    Args:
        J: max jerk (lift units/degcm^3)
        params: Dictionary containing:
            ca1: cam angle at start of segment (degcm)
            ca2t: target cam angle at end of segment (degcm)
            dca: cam angle step (degcm)
            dcaa: length of sub-segment A (degcm)
            dcab: length of sub-segment B (degcm)
            dcac: length of sub-segment C (degcm)
            dcad: length of sub-segment D (degcm)
            dcae: length of sub-segment E (degcm)
            amx: max accel limit (<lift units>/degcm^2)
    
    Returns:
        tuple: (ca2e, params)
            ca2e: cam angle error at end of segment (degcm)
            params: Updated dictionary with calculated parameters
    """
    # Store J for use in dvds2e
    params['jmx'] = J
    
    # Calculate initial cacbr (cam angle change before reversal)
    cacbr = params['amx'] / J
    
    # Get initial s2err
    s2err, params = dvds2e(cacbr, params)
    
    if s2err > 0:
        cacbrmx = cacbr
        cacbrmn = cacbr
        while s2err >= 0:
            cacbrmn = cacbrmn - params['dca']
            s2err, params = dvds2e(cacbrmn, params)
        
        # Find zero crossing using scipy's root_scalar (equivalent to MATLAB's fzero)
        sol = root_scalar(lambda x: dvds2e(x, params)[0],
                         bracket=[cacbrmn, cacbrmx],
                         method='brentq')
        cacbr = sol.root
        
    elif s2err < 0:
        cacbrmn = cacbr
        cacbrmx = cacbr
        while s2err <= 0:
            cacbrmx = cacbrmx + params['dca']
            s2err, params = dvds2e(cacbrmx, params)
        
        # Find zero crossing
        sol = root_scalar(lambda x: dvds2e(x, params)[0],
                         bracket=[cacbrmn, cacbrmx],
                         method='brentq')
        cacbr = sol.root
    
    # Calculate final cam angle error
    ca2e = (params['ca1'] + params['dcaa'] + params['dcab'] + 
            params['dcac'] + params['dcad'] + params['dcae'] - params['ca2t'])
    
    return ca2e, params 

import numpy as np
from scipy.optimize import root_scalar


def gcam0_dvd(params: dict) -> np.ndarray:
    """
    Generate cam profile for dv-d (dwell or constant velocity to dwell) segment.
    
    Args:
        params: Dictionary containing all necessary parameters
            ca1, dca, S1, S2, vr, amx, dmx, jmx, etc.
    
    Returns:
        np.ndarray: csvaj matrix with columns [ca, s, v, a, j]
    """
    # Handle rise or fall segment
    if params['S2'] > params['S1']:  # rise segment
        # Iterate J<Jmx so that end of segment is an even multiple of dca
        params['ca2t'] = 0
        ca2Jmx, _ = dvdca2e(params['jmx'], params)
        rem = ca2Jmx % params['dca']
        
        if rem == 0:
            J = params['jmx']
            ca2 = ca2Jmx
        else:
            params['ca2t'] = ca2Jmx - rem + params['dca']
            # Find first f where dvdca2e is positive
            for f in np.arange(0.9, 0, -0.1):
                if dvdca2e(f * params['jmx'], params)[0] > 0:
                    break
            
            # Find zero crossing
            sol = root_scalar(lambda x: dvdca2e(x, params)[0],
                            x0=f * params['jmx'],
                            x1=params['jmx'],
                            method='secant')
            J = sol.root
            
            params['ca2t'] = 0
            ca2, params = dvdca2e(J, params)
        
        if ca2 - params['ca1'] >= 360:
            raise ValueError('required cam angle range >= 360')
        
        # Calculate transition points
        caAB = params['ca1'] + params['dcaa']
        caBC = caAB + params['dcab']
        caCD = caBC + params['dcac']
        caDE = caCD + params['dcad']
        
        # Calculate csvaj matrix
        n = round((ca2 - params['ca1']) / params['dca']) + 1
        csvaj = np.zeros((n, 5))  # pre-allocate for compute speed
        
        for i in range(n):
            csvaj[i, 0] = params['ca1'] + params['dca'] * i
            
            if csvaj[i, 0] <= caAB:  # sub-segment A (rise): jerk=J
                carel = csvaj[i, 0] - params['ca1']
                csvaj[i, 1] = J/6 * carel**3 + params['vr'] * carel + params['S1']
                csvaj[i, 2] = J/2 * carel**2 + params['vr']
                csvaj[i, 3] = J * carel
                csvaj[i, 4] = J
                
            elif csvaj[i, 0] <= caBC and params['dcab'] > 0:  # optional sub-segment B (rise): accel=Amx
                carel = csvaj[i, 0] - caAB
                csvaj[i, 1] = params['amx']/2 * carel**2 + params['dsdcaab'] * carel + params['sab']
                csvaj[i, 2] = params['amx'] * carel + params['dsdcaab']
                csvaj[i, 3] = params['amx']
                csvaj[i, 4] = 0
                
            elif csvaj[i, 0] <= caCD:  # sub-segment C (rise): jerk=-J
                carel = csvaj[i, 0] - caBC
                csvaj[i, 1] = -J/6 * carel**3 + params['d2sdca2bc']/2 * carel**2 + params['dsdcabc'] * carel + params['sbc']
                csvaj[i, 2] = -J/2 * carel**2 + params['d2sdca2bc'] * carel + params['dsdcabc']
                csvaj[i, 3] = -J * carel + params['d2sdca2bc']
                csvaj[i, 4] = -J
                
            elif csvaj[i, 0] <= caDE and params['dcad'] > 0:  # optional sub-segment D (rise): accel=-Dmx
                carel = csvaj[i, 0] - caCD
                csvaj[i, 1] = -params['dmx']/2 * carel**2 + params['dsdcacd'] * carel + params['scd']
                csvaj[i, 2] = -params['dmx'] * carel + params['dsdcacd']
                csvaj[i, 3] = -params['dmx']
                csvaj[i, 4] = 0
                
            else:  # sub-segment E (rise): jerk=J
                carel = csvaj[i, 0] - caDE
                csvaj[i, 1] = J/6 * carel**3 + params['d2sdca2de']/2 * carel**2 + params['dsdcade'] * carel + params['sde']
                csvaj[i, 2] = J/2 * carel**2 + params['d2sdca2de'] * carel + params['dsdcade']
                csvaj[i, 3] = J * carel + params['d2sdca2de']
                csvaj[i, 4] = J
                
    else:  # fall segment
        # Swap S1 and S2 for fall calculation
        params['S1'], params['S2'] = params['S2'], params['S1']
        
        # Iterate J<Jmx so that end of segment is an even multiple of dca
        params['ca2t'] = 0
        ca2Jmx, _ = dvdca2e(params['jmx'], params)
        rem = ca2Jmx % params['dca']
        
        if rem == 0:
            J = params['jmx']
            ca2 = ca2Jmx
        else:
            params['ca2t'] = ca2Jmx - rem + params['dca']
            for f in np.arange(0.9, 0, -0.1):
                if dvdca2e(f * params['jmx'], params)[0] > 0:
                    break
            
            sol = root_scalar(lambda x: dvdca2e(x, params)[0],
                            x0=f * params['jmx'],
                            x1=params['jmx'],
                            method='secant')
            J = sol.root
            
            params['ca2t'] = 0
            ca2, params = dvdca2e(J, params)
        
        if ca2 - params['ca1'] >= 360:
            raise ValueError('required cam angle range >= 360')
        
        # Calculate transition points for fall
        caAB = ca2 - params['dcaa']
        caBC = caAB - params['dcab']
        caCD = caBC - params['dcac']
        caDE = caCD - params['dcad']
        
        # Calculate csvaj matrix
        n = round((ca2 - params['ca1']) / params['dca']) + 1
        csvaj = np.zeros((n, 5))
        
        for i in range(n-1, -1, -1):  # Reverse loop for fall segment
            csvaj[i, 0] = params['ca1'] + params['dca'] * i
            
            if csvaj[i, 0] >= caAB:  # sub-segment A (fall): jerk=-J
                carel = ca2 - csvaj[i, 0]
                csvaj[i, 1] = J/6 * carel**3 + params['vr'] * carel + params['S1']
                csvaj[i, 2] = -(J/2 * carel**2 + params['vr'])
                csvaj[i, 3] = J * carel
                csvaj[i, 4] = -J
                
            elif csvaj[i, 0] >= caBC and params['dcab'] > 0:  # optional sub-segment B (fall): accel=Amx
                carel = caAB - csvaj[i, 0]
                csvaj[i, 1] = params['amx']/2 * carel**2 + params['dsdcaab'] * carel + params['sab']
                csvaj[i, 2] = -(params['amx'] * carel + params['dsdcaab'])
                csvaj[i, 3] = params['amx']
                csvaj[i, 4] = 0
                
            elif csvaj[i, 0] >= caCD:  # sub-segment C (fall): jerk=J
                carel = caBC - csvaj[i, 0]
                csvaj[i, 1] = -J/6 * carel**3 + params['d2sdca2bc']/2 * carel**2 + params['dsdcabc'] * carel + params['sbc']
                csvaj[i, 2] = -(-J/2 * carel**2 + params['d2sdca2bc'] * carel + params['dsdcabc'])
                csvaj[i, 3] = -J * carel + params['d2sdca2bc']
                csvaj[i, 4] = J
                
            elif csvaj[i, 0] >= caDE and params['dcad'] > 0:  # optional sub-segment D (fall): accel=-Dmx
                carel = caCD - csvaj[i, 0]
                csvaj[i, 1] = -params['dmx']/2 * carel**2 + params['dsdcacd'] * carel + params['scd']
                csvaj[i, 2] = -(-params['dmx'] * carel + params['dsdcacd'])
                csvaj[i, 3] = -params['dmx']
                csvaj[i, 4] = 0
                
            else:  # sub-segment E (fall): jerk=-J
                carel = caDE - csvaj[i, 0]
                csvaj[i, 1] = J/6 * carel**3 + params['d2sdca2de']/2 * carel**2 + params['dsdcade'] * carel + params['sde']
                csvaj[i, 2] = -(J/2 * carel**2 + params['d2sdca2de'] * carel + params['dsdcade'])
                csvaj[i, 3] = J * carel + params['d2sdca2de']
                csvaj[i, 4] = -J
        
        # Restore original S1, S2 values
        params['S1'], params['S2'] = params['S2'], params['S1']
    
    return csvaj 

In [70]:
# p-p

import numpy as np


def pps2e(dcaa: float, params: dict) -> tuple[float, dict]:
    """
    Calculate lift error for p-p (peak accel to peak decel) segment.
    Matches MATLAB implementation exactly.
    """
    # Initial conditions based on dcaa
    if dcaa > 0:
        params['dcaa'] = dcaa
        params['d2sdca2ab'] = params['amx']
    else:
        params['dcaa'] = 0
        params['d2sdca2ab'] = params['amx'] + dcaa * params['jmx']
    
    # Calculate segment A end conditions
    params['sab'] = params['amx']/2 * params['dcaa']**2 + params['S1']
    params['dsdcaab'] = params['amx'] * params['dcaa']
    
    # Calculate segment B length options - exactly as in MATLAB
    dcab1 = (params['dmx'] + params['d2sdca2ab'])/params['jmx']  # d2sdca2BC=-DMX
    dcab2 = (params['d2sdca2ab']/params['jmx'] + 
             np.sqrt((params['d2sdca2ab']/params['jmx'])**2 + 
                     2*params['dsdcaab']/params['jmx']))  # dsdcaBC=0
    
    # Select minimum length for segment B
    params['dcab'] = min(dcab1, dcab2)
    
    # Calculate segment B end conditions
    params['sbc'] = (-params['jmx']/6 * params['dcab']**3 + 
                     params['d2sdca2ab']/2 * params['dcab']**2 + 
                     params['dsdcaab'] * params['dcab'] + 
                     params['sab'])
    
    params['dsdcabc'] = (-params['jmx']/2 * params['dcab']**2 + 
                         params['d2sdca2ab'] * params['dcab'] + 
                         params['dsdcaab'])
    
    # Calculate segment C length
    params['dcac'] = max(0, params['dsdcabc']/params['dmx'])
    
    # Calculate lift error
    s2e = (-params['dmx']/2 * params['dcac']**2 + 
           params['dsdcabc'] * params['dcac'] + 
           params['sbc'])/params['S2'] - 1
    
    return s2e, params


def ppca2e(J: float, params: dict) -> tuple[float, dict]:
    """
    Calculate cam angle error for p-p (peak accel to peak decel) segment.
    
    Args:
        J: jerk (<lift units>/degcm^3)
        params: Dictionary containing:
            ca1: Initial cam angle (degcm)
            ca2t: Target cam angle at end of segment (degcm)
            dca: Cam angle step (degcm)
            dcaa: Length of sub-segment A (degcm)
            dcab: Length of sub-segment B (degcm)
            dcac: Length of sub-segment C (degcm)
    
    Returns:
        tuple: (ca2e, params)
            ca2e: cam angle error at end of segment (degcm)
            params: Updated dictionary with calculated parameters
    """
    # Store J for use in pps2e
    params['jmx'] = J
    
    # Initialize dcaa
    dcaa = 0
    
    # Get initial s2err
    s2err, params = pps2e(dcaa, params)
    
    if s2err > 0:
        dcaamx = dcaa
        dcaamn = dcaa
        while s2err >= 0:
            dcaamn = dcaamn - params['dca']
            s2err, params = pps2e(dcaamn, params)
        
        # Find zero crossing using scipy's root_scalar (equivalent to MATLAB's fzero)
        sol = root_scalar(lambda x: pps2e(x, params)[0],
                         bracket=[dcaamn, dcaamx],
                         method='brentq')
        dcaa = sol.root
        
    elif s2err < 0:
        dcaamn = dcaa
        dcaamx = dcaa
        while s2err <= 0:
            dcaamx = dcaamx + params['dca']
            s2err, params = pps2e(dcaamx, params)
        
        # Find zero crossing
        sol = root_scalar(lambda x: pps2e(x, params)[0],
                         bracket=[dcaamn, dcaamx],
                         method='brentq')
        dcaa = sol.root
    
    # Calculate final cam angle error
    ca2e = params['ca1'] + params['dcaa'] + params['dcab'] + params['dcac'] - params['ca2t']
    
    return ca2e, params 

import numpy as np
from scipy.optimize import root_scalar


import numpy as np


import numpy as np


def gcam0_pp(params: dict) -> np.ndarray:
    """
    Generate cam profile for p-p (peak accel to peak decel) segment.
    """
    # Store original parameters
    J = params['jmx']
    
    # Initialize dcaa search
    dcaa = 0
    s2err, params = pps2e(dcaa, params.copy())
    
    # If s2err is negative, search forward
    if s2err < 0:
        dcaa_min = dcaa
        dcaa_step = params['dca']
        while True:
            dcaa += dcaa_step
            s2err, _ = pps2e(dcaa, params.copy())
            if s2err > 0:
                dcaa_max = dcaa
                break
            if dcaa > 100:  # Safety limit
                raise ValueError("Failed to find positive s2err")
    # If s2err is positive, search backward
    else:
        dcaa_max = dcaa
        dcaa_step = -params['dca']
        while True:
            dcaa += dcaa_step
            s2err, _ = pps2e(dcaa, params.copy())
            if s2err < 0:
                dcaa_min = dcaa
                break
            if dcaa < -100:  # Safety limit
                raise ValueError("Failed to find negative s2err")
    
    # Binary search for zero crossing with higher precision
    while abs(dcaa_max - dcaa_min) > 1e-12:  # Increased precision
        dcaa = (dcaa_min + dcaa_max) / 2
        s2err, params = pps2e(dcaa, params.copy())
        if s2err > 0:
            dcaa_max = dcaa
        else:
            dcaa_min = dcaa
    
    # Calculate final parameters with best dcaa
    _, params = pps2e(dcaa, params)
    
    # Generate csvaj matrix
    ca2 = params['ca1'] + params['dcaa'] + params['dcab'] + params['dcac']
    n = round((ca2 - params['ca1']) / params['dca']) + 1
    csvaj = np.zeros((n, 5))
    
    # Small epsilon for numerical comparisons
    eps = 1e-12
    
    for i in range(n):
        ca = params['ca1'] + params['dca'] * i
        csvaj[i, 0] = ca
        
        # Sub-segment A (optional): constant acceleration
        if ca <= params['ca1'] + params['dcaa'] + eps and params['dcaa'] > 0:
            carel = ca - params['ca1']
            csvaj[i, 1] = params['amx']/2 * carel**2 + params['S1']
            csvaj[i, 2] = params['amx'] * carel
            csvaj[i, 3] = params['amx']
            csvaj[i, 4] = 0
            
        # Sub-segment B: decreasing acceleration
        elif ca <= params['ca1'] + params['dcaa'] + params['dcab'] + eps:
            carel = ca - (params['ca1'] + params['dcaa'])
            csvaj[i, 1] = (-params['jmx']/6 * carel**3 + 
                          params['d2sdca2ab']/2 * carel**2 +
                          params['dsdcaab'] * carel + params['sab'])
            csvaj[i, 2] = (-params['jmx']/2 * carel**2 +
                          params['d2sdca2ab'] * carel + params['dsdcaab'])
            csvaj[i, 3] = -params['jmx'] * carel + params['d2sdca2ab']
            csvaj[i, 4] = -params['jmx']
            
        # Sub-segment C (optional): constant deceleration
        else:
            carel = ca - (params['ca1'] + params['dcaa'] + params['dcab'])
            csvaj[i, 1] = (-params['dmx']/2 * carel**2 +
                          params['dsdcabc'] * carel + params['sbc'])
            csvaj[i, 2] = -params['dmx'] * carel + params['dsdcabc']
            csvaj[i, 3] = -params['dmx']
            csvaj[i, 4] = 0
    
    # Clean up very small values to match MATLAB precision
    csvaj[np.abs(csvaj) < eps] = 0
    
    # Ensure final position is exactly S2
    csvaj[-1, 1] = params['S2']
    
    # Ensure final velocity is exactly 0
    if abs(csvaj[-1, 2]) < 1e-10:
        csvaj[-1, 2] = 0
    
    return csvaj

In [71]:
import numpy as np
import pandas as pd
from scipy.optimize import fsolve, root_scalar
from typing import Dict, Tuple



def gcam0(ca1: float, dca: float, s1: float, s2: float, Vr: float, Vmatch: float, 
          Amatch: float, Amx: float, Dmx: float, Jmx: float, prftype: str) -> pd.DataFrame:
    """
    Generate cam profile for various motion types.
    
    Args:
        ca1: Initial cam angle (degcm)
        dca: Cam angle step (degcm): typically 0.5 or 1.0
        s1: Initial lift (<lift units>)
        s2: Final lift (<lift units>)
        Vr: Ramp velocity (>=0, <lift units>/degcm)
        Vmatch: Final velocity for sva profile types (>=0, <lift units>/degcm)
        Amatch: Final accel for sva profile types (>=0, <lift units>/degcm^2)
        Amx: Max accel limit (<lift units>/degcm^2)
        Dmx: Max decel limit (>0, <lift units>/degcm^2)
        Jmx: Max absolute jerk limit (<lift units>/degcm^3)
        prftype: Profile type ('dv-p', 'd-v', 'dv-sva', 'dv-d', 'p-p', etc.)
    
    Returns:
        np.ndarray: csvaj matrix with columns [ca, s, v, a, j]
    """
    params = {
        # Primary parameters (matching first global line)
        'ca1': ca1,        # Initial cam angle (degcm)
        'ca2t': 0,         # Target cam angle at end of segment (degcm)
        'dca': dca,        # Cam angle step (degcm): typically 0.5 or 1.0
        'S1': s1,          # Initial lift (<lift units>)
        'S2': s2,          # Final lift (<lift units>)
        'vr': Vr,          # Ramp velocity (>=0, <lift units>/degcm)
        'vmatch': Vmatch,  # Final velocity for sva profile types (>=0, <lift units>/degcm)
        'amatch': Amatch,  # Final accel for sva profile types (>=0, <lift units>/degcm^2)
        'amx': Amx,        # Max accel limit (<lift units>/degcm^2)
        'dmx': Dmx,        # Max decel limit (>0, <lift units>/degcm^2)
        'jmx': Jmx,        # Max absolute jerk limit (<lift units>/degcm^3)
        
        # Secondary parameters (matching second global line)
        'dcaa': 0,         # Length of sub-segment A (degcm)
        'dcab': 0,         # Length of sub-segment B (degcm)
        'dcac': 0,         # Length of sub-segment C (degcm)
        'dcad': 0,         # Length of sub-segment D (degcm)
        'dcae': 0,         # Length of sub-segment E (degcm)
        'sab': 0,          # Position at end of segment A / start of B
        'sbc': 0,          # Position at end of segment B / start of C
        'scd': 0,          # Position at end of segment C / start of D
        'sde': 0,          # Position at end of segment D / start of E
        'dsdcaab': 0,      # Velocity at AB transition
        'dsdcabc': 0,      # Velocity at BC transition
        'dsdcacd': 0,      # Velocity at CD transition
        'dsdcade': 0,      # Velocity at DE transition
        'd2sdca2ab': 0,    # Acceleration at AB transition
        'd2sdca2bc': 0,    # Acceleration at BC transition
        'd2sdca2cd': 0,    # Acceleration at CD transition
        'd2sdca2de': 0     # Acceleration at DE transition
    }
    
    # Profile types:
    # 'd-v'    : dwell to constant velocity
    # 'dv-p'   : dwell or constant velocity to peak decel, peak velocity > starting velocity
    # 'dv-d'   : dwell or constant velocity to dwell, peak velocity > starting velocity
    # 'p-p'    : peak accel to peak decel
    # 'p-d'    : peak accel to dwell
    # 'dv-sva' : dwell or constant velocity to specified lift, velocity, and accel
    # 'p-sva'  : peak accel to specified lift, velocity, and accel
    
    # Generate profile based on type
    if prftype == 'dv-p':
        # dv-p: dwell or constant velocity to peak decel
        # Peak velocity must be greater than starting velocity
        csvaj = gcam0_dvp(params)
    elif prftype == 'd-v':
        # d-v: dwell to constant velocity
        csvaj = gcam0_dv(params)
    elif prftype == 'dv-sva':
        # dv-sva: dwell or constant velocity to specified velocity and acceleration
        csvaj = gcam0_dvsva(params)
    elif prftype == 'dv-d':
        # dv-d: dwell or constant velocity to dwell
        csvaj = gcam0_dvd(params)
    elif prftype == 'p-p':
        # p-p: peak accel to peak decel
        csvaj = gcam0_pp(params)
    else:
        raise ValueError(f'Unknown profile type: {prftype}')
    
    return pd.DataFrame(csvaj, columns=['ca', 's', 'v', 'a', 'j']) 

In [72]:
import numpy as np


# Test case matching MATLAB output
test_params = {
    'ca1': 100.0,
    'dca': 0.5,
    'S1': 2.0,
    'S2': 3.0,
    'vr': 0.0,  # Starting with 0 velocity
    'vmatch': 0.05,
    'amatch': 0.005,
    'amx': 0.005,
    'dmx': 0.005,
    'jmx': 0.005
}

print("Initial parameter values:")
for key in ['S1', 'S2', 'amx', 'dmx', 'jmx', 'vr']:
    print(f"{key:4s} = {test_params[key]}")
print("\n")

# Initial test
params = test_params.copy()
s2err, params = pps2e(0, params)
print(f"With dcaa = 0:")
print(f"s2err = {s2err}")
print(f"dcaa = {params['dcaa']}")
print(f"dcab = {params['dcab']}")
print(f"dcac = {params['dcac']}")
print(f"dsdcaab = {params['dsdcaab']}")
print(f"dsdcabc = {params['dsdcabc']}")
print(f"d2sdca2ab = {params['d2sdca2ab']}")

print("\nTesting range of dcaa values:")
print("dcaa        s2err         dcab         dcac")
print("-------------------------------------------")
for dcaa in [-0.2, -0.1, 0, 0.1, 0.2]:
    params = test_params.copy()
    s2err, params = pps2e(dcaa, params)
    print(f"{dcaa:5.1f}    {s2err:10.6f}    {params['dcab']:10.6f}    {params['dcac']:10.6f}")

Initial parameter values:
S1   = 2.0
S2   = 3.0
amx  = 0.005
dmx  = 0.005
jmx  = 0.005
vr   = 0.0


With dcaa = 0:
s2err = -0.3322222222222222
dcaa = 0
dcab = 2.0
dcac = 0
dsdcaab = 0.0
dsdcabc = 0.0
d2sdca2ab = 0.005

Testing range of dcaa values:
dcaa        s2err         dcab         dcac
-------------------------------------------
 -0.2     -0.332764      1.600000      0.000000
 -0.1     -0.332523      1.800000      0.000000
  0.0     -0.332222      2.000000      0.000000
  0.1     -0.331872      2.000000      0.100000
  0.2     -0.331489      2.000000      0.200000


In [73]:
gcam0(100, 0.5, 2, 3, 0.05, 0.05, 0.005, 0.005, 0.005, 0.005, 'p-p')

,ca,s,v,a,j
0,100.0,2.000000,0.000000,0.00500,0.000
1,100.5,2.000625,0.002500,0.00500,0.000
2,101.0,2.002500,0.005000,0.00500,0.000
3,101.5,2.005625,0.007500,0.00500,0.000
4,102.0,2.010000,0.010000,0.00500,0.000
5,102.5,2.015625,0.012500,0.00500,0.000
6,103.0,2.022500,0.015000,0.00500,0.000
7,103.5,2.030625,0.017500,0.00500,0.000
8,104.0,2.040000,0.020000,0.00500,0.000
9,104.5,2.050625,0.022500,0.00500,0.000
